## IN THIS NOTEBOOK, WE WILL EXPLORE THE COLBERT AS A RERANKER AND RETRIEVER IN LOCAL MODE. 

* If you want to build a server from your colbert local index, please refer [here](https://github.com/stanford-futuredata/ColBERT/blob/main/server.py)

In [1]:
from colbert.infra.config import ColBERTConfig

print(ColBERTConfig())

/home/athekunal/DSPy-contributions/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ColBERTConfig(query_token_id='[unused0]', doc_token_id='[unused1]', query_token='[Q]', doc_token='[D]', ncells=None, centroid_score_threshold=None, ndocs=None, load_index_with_mmap=False, index_path=None, index_bsize=64, nbits=1, kmeans_niters=4, resume=False, similarity='cosine', bsize=32, accumsteps=1, lr=3e-06, maxsteps=500000, save_every=None, warmup=None, warmup_bert=None, relu=False, nway=2, use_ib_negatives=False, reranker=False, distillation_alpha=1.0, ignore_scores=False, model_name=None, query_maxlen=32, attend_to_mask_tokens=False, interaction='colbert', dim=128, doc_maxlen=220, mask_punctuation=True, checkpoint=None, triples=None, collection=None, queries=None, index_name=None, overwrite=False, root='/home/athekunal/DSPy-contributions/dspy/examples/integrations/colbert/experiments', experiment='default', index_root=None, name='2024-04/09/20.41.14', rank=0, nranks=1, amp=True, gpus=1, avoid_fork_if_possible=False)


## Let's review the colbert config class

In [2]:
for k,v in ColBERTConfig().__dict__.items():
    print(f"{k} --> {v}")

query_token_id --> [unused0]
doc_token_id --> [unused1]
query_token --> [Q]
doc_token --> [D]
ncells --> None
centroid_score_threshold --> None
ndocs --> None
load_index_with_mmap --> False
index_path --> None
index_bsize --> 64
nbits --> 1
kmeans_niters --> 4
resume --> False
similarity --> cosine
bsize --> 32
accumsteps --> 1
lr --> 3e-06
maxsteps --> 500000
save_every --> None
warmup --> None
warmup_bert --> None
relu --> False
nway --> 2
use_ib_negatives --> False
reranker --> False
distillation_alpha --> 1.0
ignore_scores --> False
model_name --> None
query_maxlen --> 32
attend_to_mask_tokens --> False
interaction --> colbert
dim --> 128
doc_maxlen --> 220
mask_punctuation --> True
checkpoint --> None
triples --> None
collection --> None
queries --> None
index_name --> None
overwrite --> False
root --> /home/athekunal/DSPy-contributions/dspy/examples/integrations/colbert/experiments
experiment --> default
index_root --> None
name --> 2024-04/09/20.41.14
rank --> 0
nranks --> 1
amp

In [3]:
passages =  ["It's a piece of cake.", "Don't put off until tomorrow what you can do today.", 'To kill two birds with one stone.', 'Actions speak louder than words.', 'Honesty is the best policy.', 'If you want something done right, do it yourself.', 'The best things in life are free.', "Don't count your chickens before they hatch.", 'She sells seashells by the seashore.', 'Practice makes perfect.', "Where there's a will, there's a way.", 'Absence makes the heart grow fonder.', 'When the going gets tough, the tough get going.', 'A journey of a thousand miles begins with a single step.', "You can't have your cake and eat it too.", "If you can't beat them, join them.", 'Keep your friends close and your enemies closer.', "Don't put all your eggs in one basket.", "All's fair in love and war.", 'Every dog has its day.', 'All good things must come to an end.', 'Once bitten, twice shy.', "The apple doesn't fall far from the tree.", 'A penny saved is a penny earned.', "Don't bite the hand that feeds you.", 'You reap what you sow.', 'An apple a day keeps the doctor away.', "One man's trash is another man's treasure.", 'The squeaky wheel gets the grease.', 'A picture is worth a thousand words.', 'Fortune favors the bold.', 'Practice what you preach.', 'A watched pot never boils.', 'No pain, no gain.', "You can't make an omelet without breaking eggs.", "There's no place like home.", 'Ask and you shall receive.', 'Let sleeping dogs lie.', 'If the shoe fits, wear it.', 'Every cloud has a silver lining.', 'Look before you leap.', 'The more, the merrier.', 'The grass is always greener on the other side.', 'Beauty is only skin deep.', "Two wrongs don't make a right.", 'Beauty is in the eye of the beholder.', 'Necessity is the mother of invention.', 'Out of sight, out of mind.', 'Patience is a virtue.', 'Curiosity killed the cat.', "If at first you don't succeed, try, try again.", "Beggars can't be choosers.", 'Too many cooks spoil the broth.', 'Easy come, easy go.', "Don't cry over spilled milk.", "There's no such thing as a free lunch.", 'A bird in the hand is worth two in the bush.', 'Good things come to those who wait.', 'The quick brown fox jumps over the lazy dog.', 'It takes two to tango.', 'A friend in need is a friend indeed.', 'Like father, like son.', 'Let bygones be bygones.', 'Kill two birds with one stone.', 'A penny for your thoughts.', 'I am the master of my fate, I am the captain of my soul.', 'The pen is mightier than the sword.', 'When in Rome, do as the Romans do.', "Rome wasn't built in a day.", "You can't judge a book by its cover.", "It's raining cats and dogs.", 'Make hay while the sun shines.', "It's better to be safe than sorry.", 'The early bird catches the worm.', 'To be or not to be, that is the question.', 'Better late than never.']

## This tutorial is running from the examples/integrations/tutorials folder, hence we need to add the system path for dspy

In [4]:
import sys
sys.path.append("../../..")

## COLBERT AS RETRIEVER

In [7]:
import dspy
colbert_config = ColBERTConfig()
colbert_retriever = dspy.ColBERTv2RetrieverLocal(
    checkpoint='colbert-ir/colbertv2.0',passages = passages,load_only=False,
    index_name='colbert-ir-index',colbert_config=colbert_config
)

Building the index for experiment default with index name colbert-ir-index


[Apr 09, 20:41:33] #> Creating directory /home/athekunal/DSPy-contributions/dspy/examples/integrations/colbert/experiments/default/indexes/colbert-ir-index 


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 1,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "query_maxlen": 32,


WARNING clustering 728 points to 256 centroids: please provide at least 9984 training points
Using /home/athekunal/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/athekunal/.cache/torch_extensions/py310_cu117/decompress_residuals_cpp/build.ninja...
Building extension module decompress_residuals_cpp...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module decompress_residuals_cpp...


[Apr 09, 20:41:39] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
ninja: no work to do.
[0.053, 0.05, 0.061, 0.053, 0.044, 0.06, 0.047, 0.049, 0.064, 0.05, 0.051, 0.057, 0.052, 0.042, 0.061, 0.069, 0.052, 0.051, 0.063, 0.043, 0.04, 0.052, 0.043, 0.042, 0.035, 0.055, 0.07, 0.048, 0.051, 0.04, 0.057, 0.045, 0.054, 0.052, 0.042, 0.051, 0.048, 0.047, 0.056, 0.059, 0.05, 0.063, 0.054, 0.06, 0.046, 0.051, 0.04, 0.071, 0.04, 0.049, 0.056, 0.043, 0.048, 0.051, 0.045, 0.052, 0.041, 0.073, 0.039, 0.045, 0.052, 0.056, 0.053, 0.06, 0.041, 0.053, 0.054, 0.052, 0.051, 0.05, 0.061, 0.053, 0.035, 0.05, 0.049, 0.057, 0.045, 0.044, 0.05, 0.05, 0.041, 0.048, 0.043, 0.049, 0.05, 0.039, 0.056, 0.055, 0.048, 0.045, 0.044, 0.041, 0.046, 0.044, 0.046, 0.064, 0.056, 0.054, 0.058, 0.04, 0.043, 0.045, 0.051, 0.058, 0.06, 0.043, 0.057, 0.043, 0.053, 0.056, 0.047, 0.039, 0.057, 0.044, 0.055, 0.063, 0.041, 0.047, 0.049, 0.051, 0.046, 0.042, 0.053, 0.045, 0.044, 0.053

Using /home/athekunal/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/athekunal/.cache/torch_extensions/py310_cu117/packbits_cpp/build.ninja...
Building extension module packbits_cpp...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module packbits_cpp...
0it [00:00, ?it/s]

[Apr 09, 20:41:40] [0] 		 #> Saving chunk 0: 	 76 passages and 766 embeddings. From #0 onward.
[Apr 09, 20:41:40] [0] 		 #> Checking all files were saved...
[Apr 09, 20:41:40] [0] 		 Found all files!
[Apr 09, 20:41:40] [0] 		 #> Building IVF...
[Apr 09, 20:41:40] [0] 		 #> Loading codes...
[Apr 09, 20:41:40] [0] 		 Sorting codes...
[Apr 09, 20:41:40] [0] 		 Getting unique codes...
[Apr 09, 20:41:40] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 09, 20:41:40] #> Building the emb2pid mapping..
[Apr 09, 20:41:40] len(emb2pid) = 766
[Apr 09, 20:41:40] #> Saved optimized IVF to /home/athekunal/DSPy-contributions/dspy/examples/integrations/colbert/experiments/default/indexes/colbert-ir-index/ivf.pid.pt
[Apr 09, 20:41:40] [0] 		 #> Saving the indexing metadata to /home/athekunal/DSPy-contributions/dspy/examples/integrations/colbert/experiments/default/indexes/colbert-ir-index/metadata.json ..


...

Loading extension module decompress_residuals_cpp...
Using /home/athekunal/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/athekunal/.cache/torch_extensions/py310_cu117/packbits_cpp/build.ninja...
Building extension module packbits_cpp...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
[Apr 09, 20:41:43] #> Loading IVF...
[Apr 09, 20:41:43] #> Loading doclens...


Loading extension module packbits_cpp...
100%|██████████| 1/1 [00:00<00:00, 2012.62it/s]

[Apr 09, 20:41:43] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 722.16it/s]


In [8]:
#CONFIGURE COLBERT IN DSPY
dspy.settings.configure(rm=colbert_retriever)

retrieved_docs = dspy.Retrieve(k=5)

In [10]:
pred = retrieved_docs(
    "What is the meaning of life?"
)

/home/athekunal/DSPy-contributions/dspy/examples/integrations/colbert/../../../dsp/primitives/search.py:74: UserWarning: If you want to use the Reranker, please use dspy.RetrieveThenRerank. The reranking is ignored here.
  warnings.warn("If you want to use the Reranker, please use dspy.RetrieveThenRerank. The reranking is ignored here.")
/home/athekunal/DSPy-contributions/dspy/examples/integrations/colbert/../../../dsp/primitives/search.py:13: UserWarning: If you want to use the Reranker, please use dspy.RetrieveThenRerank
  warnings.warn("If you want to use the Reranker, please use dspy.RetrieveThenRerank")


In [11]:
pred

Prediction(
    score=[nan, nan, nan, nan, nan],
    pid=[33, 6, 47, 74, 48],
    passages=['No pain, no gain.', 'The best things in life are free.', 'Out of sight, out of mind.', 'To be or not to be, that is the question.', 'Patience is a virtue.']
)

## COLBERT AS RERANKER

In [14]:
colbert_config = ColBERTConfig()
colbert_config.index_name = 'colbert-ir-index'
colbert_reranker = dspy.ColBERTv2RerankerLocal(
    checkpoint='colbert-ir/colbertv2.0',colbert_config=colbert_config)

In [15]:
dspy.settings.configure(rm=colbert_retriever,reranker=colbert_reranker)

retrieve_rerank = dspy.RetrieveThenRerank(k=5)

In [17]:
pred = retrieve_rerank(
    ["What is the meaning of life?","Meaning of pain?"]
)

In [18]:
pred

[Prediction(
     score=[nan, nan, nan, nan, nan],
     pid=[6, 48, 74, 47, 33],
     rerank_score=[15.8359375, 14.2109375, 12.5703125, 11.7890625, 9.1796875],
     passages=['The best things in life are free.', 'Patience is a virtue.', 'To be or not to be, that is the question.', 'Out of sight, out of mind.', 'No pain, no gain.']
 ),
 Prediction(
     score=[nan, nan, nan, nan, nan],
     pid=[33, 0, 47, 74, 16],
     rerank_score=[19.828125, 12.2890625, 11.171875, 9.09375, 6.8984375],
     passages=['No pain, no gain.', "It's a piece of cake.", 'Out of sight, out of mind.', 'To be or not to be, that is the question.', 'Keep your friends close and your enemies closer.']
 )]

## YOU CAN ALSO COLBERT RERANKER AS STANDALONE MODEL

In [25]:
import numpy as np
import tabulate

scores_arr = colbert_reranker(
    "What is the meaning of life and pain?",
    passages
)

tabulate_data = []
for idx in np.argsort(scores_arr)[::-1]:
    # print(f"Passage = {passages[idx]} --> Score = {scores_arr[idx]}")
    tabulate_data.append([passages[idx],scores_arr[idx]])

table = tabulate.tabulate(tabulate_data,tablefmt="html",headers={'sentence','score'})

In [26]:
from IPython.display import HTML, display
display(HTML(table))

score,sentence
"No pain, no gain.",16.4844
The best things in life are free.,12.5156
Patience is a virtue.,11.7578
It's better to be safe than sorry.,11.2109
A friend in need is a friend indeed.,10.4609
"Out of sight, out of mind.",10.3516
"Once bitten, twice shy.",10
It's a piece of cake.,10
All's fair in love and war.,9.92188
All good things must come to an end.,9.92188
